In [ ]:
import time

import pandas as pd
import polars as pl
import yfinance as yf
from modules.investment_info_fetcher import investment_info_fetcher
from modules.get_environment_variable import get_environment_variable

connection_config: dict[str, str | None] = get_environment_variable()

In [4]:
df_investment_info: pl.DataFrame = investment_info_fetcher(DB_HOST, DB_PORT, DB_USER, DB_PASSWORD, DB_NAME)

In [47]:
yf_tickers = yf.Tickers(tickers=['VTI', 'VGK'])
df_yf = yf_tickers.history(period='1y', interval='1d')['Dividends']

df_yf = (
    pl.DataFrame(df_yf.reset_index())
    .rename({"Date": "date"})
    .with_columns(pl.col("date").cast(pl.Date))
)

yf_ticker = yf.Ticker('JPY=X')
df_yf_jpy = yf_ticker.history(period='1y', interval='1d')['Close']

df_yf_jpy = (
    pl.DataFrame(df_yf_jpy.reset_index())
    .rename({"Date": "date"})
    .with_columns(pl.col("date").cast(pl.Date))
)

df = df_yf.join(df_yf_jpy, on='date', how='inner')
df.filter(pl.col('VGK') > 0)

[*********************100%***********************]  2 of 2 completed


date,VGK,VTI,Close
date,f64,f64,f64
2024-03-15,0.37,0.0,148.296005
2024-12-20,0.719,0.0,157.643997


In [ ]:
yf_ticker = yf.Ticker('JPY=X')
exchange_rate = yf_ticker.history(period='max')['Close']
df_exchange_rate = pl.DataFrame(exchange_rate.reset_index()).rename({"Date": "date"}).with_columns(pl.col("date").cast(pl.Date))

start_date = df_exchange_rate['date'].min()
end_date = df_exchange_rate['date'].max()

date_range = pd.date_range(start=start_date, end=end_date, freq='D')
df_date = (
    pl.DataFrame(date_range.to_numpy())
    # 名前をつける
    .rename({"column_0": "date"})
    # 日付型に変換
    .with_columns(pl.col("date").cast(pl.Date))
)
df_date
df_exchange_rate = df_date.join(df_exchange_rate, on='date', how='left')
print(df_exchange_rate)
df_exchange_rate.fill_null(strategy='forward')

shape: (10_365, 2)
┌────────────┬────────────┐
│ date       ┆ Close      │
│ ---        ┆ ---        │
│ date       ┆ f64        │
╞════════════╪════════════╡
│ 1996-10-30 ┆ 114.18     │
│ 1996-10-31 ┆ null       │
│ 1996-11-01 ┆ 113.5      │
│ 1996-11-02 ┆ null       │
│ 1996-11-03 ┆ null       │
│ …          ┆ …          │
│ 2025-03-12 ┆ 148.042007 │
│ 2025-03-13 ┆ 148.279007 │
│ 2025-03-14 ┆ 147.949005 │
│ 2025-03-15 ┆ null       │
│ 2025-03-16 ┆ 148.610001 │
└────────────┴────────────┘


date,Close
u32,u32
0,0
